In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
import src.slurm_cluster as scluster
# client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val
 

In [ ]:
# Input the MMEM annual mean SAT data
input_model = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Datafiles/'

EC_Earth_data = xr.open_mfdataset(input_model + 'tas_EC_Earth_annual_ano_1850_2022.nc',chunks = {'run':1})

In [ ]:
EC_Earth_data

### Apply the MMEs GSAT timeseries to extract forced and unforced component of GSAT

In [ ]:
# input GSAT time series
input_ts_dir = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure1/'

GSAT_EC_Earth_annual = xr.open_dataset(input_ts_dir + 'GMSAT_EC_Earth_annual_timeseries_ENS.nc')
GSAT_EC_Earth_annual

In [ ]:
EC_Earth_data['tas'].values.shape

In [ ]:
slope_Forced_signal, intercept_Forced_signal = data_process.linear_regression_single_gmst_multi_sat(
    GSAT_EC_Earth_annual['tas'],
    EC_Earth_data['tas'].values)

In [ ]:
# separate the GSAT into forced and internal variability
lat = EC_Earth_data['lat']
lon = EC_Earth_data['lon']
run = EC_Earth_data['run']

In [ ]:
slope_Forced_signal.shape

In [ ]:
slope_Forced_signal_da = xr.DataArray(slope_Forced_signal, coords=[run,lat, lon], dims=['run','lat', 'lon'])
intercept_Forced_signal_da = xr.DataArray(intercept_Forced_signal, coords=[run, lat, lon], dims=['run','lat', 'lon'])

In [ ]:
intercept_Forced_signal_da

In [ ]:
# reconstruct the forced signal
GSAT_EC_Earth_Forced_signal = GSAT_EC_Earth_annual['tas'] * slope_Forced_signal_da  + intercept_Forced_signal_da


In [ ]:
# reorder the dimensions
GSAT_EC_Earth_Forced_signal = GSAT_EC_Earth_Forced_signal.transpose('run','year','lat','lon') 

In [ ]:
GSAT_EC_Earth_Forced_signal

In [ ]:
# calculate the internal variability
GSAT_EC_Earth_Internal_variability = EC_Earth_data - GSAT_EC_Earth_Forced_signal

In [ ]:
GSAT_EC_Earth_Internal_variability['tas'].min().values

In [ ]:
# output the forced and unforced anomalies
output_dir = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure2/EC_Earth/'

GSAT_EC_Earth_Forced_signal.to_netcdf(output_dir + 'GSAT_EC_Earth_Forced_anomalies_1850_2022.nc')
GSAT_EC_Earth_Internal_variability.to_netcdf(output_dir + 'GSAT_EC_Earth_Internal_Variability_anomalies_1850_2022.nc')